In [1]:
#Edgar Moises Hernandez-Gonzalez
#08/07/19-10/07/19
#Clasificar EEG de habla imaginada con CNN y LSTM, con CV=5

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [2]:
datos = pd.read_csv("TodoSubmues01.csv", header=None)
x = datos.iloc[:, 0:3072].values
y = datos.iloc[:, 3072].values

num_classes = 6
n_samples = len(y)
print(x.shape)
print(y.shape)
print(n_samples)

(304, 3072)
(304,)
304


In [3]:
#escalar los datos para que tengan media = 0 y desviacion estandar = 1
scaler = StandardScaler().fit(x)
x = scaler.transform(x)

In [4]:
#convertir de 2D a 4D
x = x.reshape((n_samples, 1, 512, 6))
print(x.shape)

(304, 1, 512, 6)


In [5]:
skf = StratifiedKFold(n_splits = 5)

In [6]:
resultados = np.zeros((2, 5)) #accuracy y loss en cada validacion cruzada
i = 0
for iTrain, jTest in skf.split(x, y):
    print("\nIteracion:", i + 1)
    x_train, x_test = x[iTrain], x[jTest]
    y_train, y_test = y[iTrain], y[jTest]
    
    model = models.Sequential()
    model.add(layers.TimeDistributed(layers.Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,512,6)))
    model.add(layers.TimeDistributed(layers.Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(layers.TimeDistributed(layers.Dropout(0.5)))
    model.add(layers.TimeDistributed(layers.MaxPooling1D(pool_size=2)))
    model.add(layers.TimeDistributed(layers.Flatten()))
    model.add(layers.LSTM(100))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(x_train, y_train, epochs = 20, batch_size = 64, validation_data=(x_test, y_test))
    
    resultados[0, i], resultados[1, i] = model.evaluate(x_test, y_test)
    i += 1


Iteracion: 1
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 239 samples, validate on 65 samples
Epoch 1/20
239/239 [==============================] - 3s 12ms/sample - loss: 1.8387 - acc: 0.1506 - val_loss: 1.7970 - val_acc: 0.1385
Epoch 2/20
239/239 [==============================] - 0s 1ms/sample - loss: 1.8222 - acc: 0.1715 - val_loss: 1.8021 - val_acc: 0.1231
Epoch 3/20
239/239 [==============================] - 0s 1ms/sample - loss: 1.7924 - acc: 0.2134 - val_loss: 1.8004 - val_acc: 0.1846
Epoch 4/20
239/239 [==============================] - 0s 1ms/sample - loss: 1.7400 - acc: 0.2762 - val_loss: 1.8052 - val_acc: 0.1077
Epoch 5/20
239/239 [==============================] - 0s 1ms/sample - loss: 1.7183 - acc: 0.2469 - val_loss: 1.8025 - val_acc: 0.1385
Epoch 6/20
239/239 [==============================] - 0s 1ms/sample - loss: 1.6503 

244/244 [==============================] - 1s 3ms/sample - loss: 0.9286 - acc: 0.7828 - val_loss: 1.9621 - val_acc: 0.1667
Epoch 15/20
244/244 [==============================] - 0s 1ms/sample - loss: 0.8361 - acc: 0.8484 - val_loss: 2.0252 - val_acc: 0.2000
Epoch 16/20
244/244 [==============================] - 0s 1ms/sample - loss: 0.7114 - acc: 0.8893 - val_loss: 2.0208 - val_acc: 0.2167
Epoch 17/20
244/244 [==============================] - 0s 1ms/sample - loss: 0.6540 - acc: 0.8648 - val_loss: 2.1007 - val_acc: 0.2500
Epoch 18/20
244/244 [==============================] - 0s 2ms/sample - loss: 0.5677 - acc: 0.9057 - val_loss: 2.0968 - val_acc: 0.2000
Epoch 19/20
244/244 [==============================] - 1s 3ms/sample - loss: 0.4634 - acc: 0.9180 - val_loss: 2.1953 - val_acc: 0.2000
Epoch 20/20
60/60 [==============================] - 0s 299us/sample - loss: 2.2601 - acc: 0.2333

Iteracion: 4
Train on 244 samples, validate on 60 samples
Epoch 1/20
244/244 [=========================

In [7]:
print("loss:", resultados[0])
print("accuracy:", resultados[1])
print("mean accuracy:", resultados[1].mean())

loss: [2.34861025 2.3126816  2.26005395 2.26888148 2.44756043]
accuracy: [0.24615385 0.11666667 0.23333333 0.15000001 0.13559322]
mean accuracy: 0.1763494148850441
